In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')

# Fully connected layer

### Fill this to continue running smoothly, I think

In [ ]:
x_train = 
y_train = 
x_val =
y_val
no_epoch = 
batch_size = 
img_height =
img_width =
channels = 

### Importing needed packages

In [ ]:
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import models
from keras import layers

## Building initial model

In [ ]:
FC_init = models.Sequential()
FC_init.add(layers.Dense(512, activation='relu', name='Layer_1', input_shape=(img_width * img_height * channels,)))
FC_init.add(layers.Dense(256, activation='relu', name='Layer_2'))
FC_init.add(layers.Dense(128, activation='relu', name='Layer_3'))
FC_init.add(layers.Dense(10, activation='softmax'))

In [ ]:
opt = keras.optimizers.Adam(learning_rate=10e-3)
FC_init.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
FC_init.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)


In [ ]:
History = FC_init.fit(x_train,y_train, validation_data=(x_val,y_val), verbose = 1, epochs = no_epoch, batch_size = batch_size,callbacks=[early_stopping])

In [ ]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']
loss = History.history['loss']
val_loss = History.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

---
## Building another model